## Script created to create a negative list to help ensure reproducibility
Previous method caused to have different negative examples that kept changingin the performance results which made it hard to compare my models. So, instead I took the method of getting negatives separately so the main script is consistent when the Logistic Regression model is trained. 

Still implemented the same method of picking random negatives and the same number of negative examples as there are positive examples. Also, the negatives can't be in the positive set. The final negative set is saved as a tsv file and read in the load_negatives funciton of the main scirpt. 
## **DO NOT RUN**

In [16]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import time
from scipy import stats
from scipy.stats import norm
import random
from sklearn.preprocessing import StandardScaler


In [17]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [18]:
# read in network data file
biogrid_raw = pd.read_csv("../data/bigdatasets/biogrid_network.txt", sep='\t') 

#take a look
biogrid_raw.head()


,#ID Interactor A,ID Interactor B,Alt IDs Interactor A,Alt IDs Interactor B,Aliases Interactor A,Aliases Interactor B,Interaction Detection Method,Publication 1st Author,Publication Identifiers,Taxid Interactor A,Taxid Interactor B,Interaction Types,Source Database,Interaction Identifiers,Confidence Values
0,entrez gene/locuslink:6416,entrez gene/locuslink:2318,biogrid:112315|entrez gene/locuslink:MAP2K4|un...,biogrid:108607|entrez gene/locuslink:FLNC|unip...,entrez gene/locuslink:JNKK(gene name synonym)|...,entrez gene/locuslink:ABP-280(gene name synony...,"psi-mi:""MI:0018""(two hybrid)",Marti A (1997),pubmed:9006895,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:103,-
1,entrez gene/locuslink:84665,entrez gene/locuslink:88,biogrid:124185|entrez gene/locuslink:MYPN|unip...,biogrid:106603|entrez gene/locuslink:ACTN2|uni...,entrez gene/locuslink:CMD1DD(gene name synonym...,entrez gene/locuslink:CMD1AA(gene name synonym),"psi-mi:""MI:0018""(two hybrid)",Bang ML (2001),pubmed:11309420,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:117,-
2,entrez gene/locuslink:90,entrez gene/locuslink:2339,biogrid:106605|entrez gene/locuslink:ACVR1|uni...,biogrid:108625|entrez gene/locuslink:FNTA|unip...,entrez gene/locuslink:ACTRI(gene name synonym)...,entrez gene/locuslink:FPTA(gene name synonym)|...,"psi-mi:""MI:0018""(two hybrid)",Wang T (1996),pubmed:8599089,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:183,-
3,entrez gene/locuslink:2624,entrez gene/locuslink:5371,biogrid:108894|entrez gene/locuslink:GATA2|uni...,biogrid:111384|entrez gene/locuslink:PML|unipr...,entrez gene/locuslink:DCML(gene name synonym)|...,entrez gene/locuslink:MYL(gene name synonym)|e...,"psi-mi:""MI:0018""(two hybrid)",Tsuzuki S (2000),pubmed:10938104,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:278,-
4,entrez gene/locuslink:6118,entrez gene/locuslink:6774,biogrid:112038|entrez gene/locuslink:RPA2|entr...,biogrid:112651|entrez gene/locuslink:STAT3|uni...,entrez gene/locuslink:REPA2(gene name synonym)...,entrez gene/locuslink:ADMIO(gene name synonym)...,"psi-mi:""MI:0018""(two hybrid)",Kim J (2000),pubmed:10875894,taxid:9606,taxid:9606,"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0463""(biogrid)",biogrid:418,-


In [19]:
# biogrid has proteins for other species. I want only the human proteins so filter these out.
human_biogrid_raw =biogrid_raw[(biogrid_raw['Taxid Interactor A'] == "taxid:9606") & (biogrid_raw['Taxid Interactor B'] == "taxid:9606")]

In [20]:
# get rid of umwanted columns
columns_to_drop= ['Alt IDs Interactor A',
 'Alt IDs Interactor B',
 'Aliases Interactor A',
 'Aliases Interactor B',
 'Interaction Detection Method',
 'Publication 1st Author',
 'Publication Identifiers',
 'Taxid Interactor A',
 'Taxid Interactor B','Interaction Types',
 'Source Database',
 'Interaction Identifiers']
human_biogrid_raw.drop(columns=columns_to_drop, inplace=True)

human_biogrid_raw.head()
print("Shape of dataframe", human_biogrid_raw.shape)

Shape of dataframe (1131384, 3)


/var/folders/9q/tl4z5rk169514hgp4blpglqc0000gn/T/ipykernel_51524/1794002502.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  human_biogrid_raw.drop(columns=columns_to_drop, inplace=True)


In [21]:
# format values in the confidence values so only the numeric part is kept

#checking for na scores. NA scores are represented with '-'
look_at_score =human_biogrid_raw["Confidence Values"].unique()
# for score in look_at_score:
#     print(score)

#remove the score part
human_biogrid_raw.loc[:,'Confidence Values'] = human_biogrid_raw['Confidence Values'].str.replace('score:', '') 
    
print(human_biogrid_raw.dtypes)
#get rid of any rows that have '-' in the confidence value
filtered_biogrid = human_biogrid_raw[human_biogrid_raw['Confidence Values'] != "-"]

# ensure correct data type for the confidence scores.
filtered_biogrid['Confidence Values'] = filtered_biogrid['Confidence Values'].astype(float)
print("After changing dat types \n", filtered_biogrid.dtypes)

# now doing the same for the genes in interactor a and b columns
filtered_biogrid.loc[:, '#ID Interactor A'] = filtered_biogrid['#ID Interactor A'].str.split(':').str[-1]

filtered_biogrid.loc[:, 'ID Interactor B'] = filtered_biogrid['ID Interactor B'].str.split(':').str[-1]

# check data type of columns
print("Check data type\n", filtered_biogrid.dtypes)

#ID Interactor A     object
ID Interactor B      object
Confidence Values    object
dtype: object
After changing dat types 
 #ID Interactor A      object
ID Interactor B       object
Confidence Values    float64
dtype: object


/var/folders/9q/tl4z5rk169514hgp4blpglqc0000gn/T/ipykernel_51524/1519717752.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_biogrid['Confidence Values'] = filtered_biogrid['Confidence Values'].astype(float)


Check data type
 #ID Interactor A      object
ID Interactor B       object
Confidence Values    float64
dtype: object


In [22]:
# check data frame
filtered_biogrid.head()

,#ID Interactor A,ID Interactor B,Confidence Values
121446,10421,23020,1.000
121447,10755,4646,0.952
121448,1974,1981,0.931
121449,10421,10594,0.919
121450,382,10128,0.909


In [23]:
# check range of confidence values
max_conf_values = filtered_biogrid['Confidence Values'].max()
min_conf_values = filtered_biogrid['Confidence Values'].min()

print(f"Max confidence value { max_conf_values} and min confidence value {min_conf_values}")

Max confidence value 52530.0 and min confidence value -24.621


In [24]:
# Scale confidence values to improve model performance

scale = StandardScaler()
filtered_biogrid['Standardized_confidence_values'] = scale.fit_transform(filtered_biogrid['Confidence Values'].values.reshape(-1, 1))
# drop confidence values column and only keep the standardized values
filtered_biogrid.drop(columns='Confidence Values', inplace=True)

filtered_biogrid.head(20)

/var/folders/9q/tl4z5rk169514hgp4blpglqc0000gn/T/ipykernel_51524/4269995461.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_biogrid['Standardized_confidence_values'] = scale.fit_transform(filtered_biogrid['Confidence Values'].values.reshape(-1, 1))
/var/folders/9q/tl4z5rk169514hgp4blpglqc0000gn/T/ipykernel_51524/4269995461.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_biogrid.drop(columns='Confidence Values', inplace=True)


,#ID Interactor A,ID Interactor B,Standardized_confidence_values
121446,10421,23020,-0.070923
121447,10755,4646,-0.071073
121448,1974,1981,-0.071139
121449,10421,10594,-0.071177
121450,382,10128,-0.071208
121451,7532,3799,-0.071236
121452,9337,23019,-0.071239
121453,55723,79711,-0.071568
121454,28988,8924,-0.071603
121455,9410,10594,-0.071612


In [25]:
# need to get the adjecancy matrix of the biogrid data frame

# first check to see if the edge list results in a connected graph
graph = nx.from_pandas_edgelist(filtered_biogrid, '#ID Interactor A', 'ID Interactor B')
is_connected = nx.is_connected(graph)
print(is_connected)
# output whether the graph is connected or not
if is_connected:
    print("The Human BioGRID network is fully connected.")
else:
    print("The Human BioGRID network is not fully connected.")

# find the connected components
connected_components = list(nx.connected_components(graph))

# get the largest connected component
largest_component = max(connected_components, key=len)
print(f"Largest component size: {len(largest_component)}.")

# keep only interactions within the largest connected component
only_connected_biogrid = filtered_biogrid[
    (filtered_biogrid['#ID Interactor A'].isin(largest_component)) &
    (filtered_biogrid['ID Interactor B'].isin(largest_component))
]

# start building the adjacency matrix
num_genes = len(largest_component)
adj_matrix = np.zeros((num_genes, num_genes))

# create a mapping of genes to indices
gene_to_index = {gene: i for i, gene in enumerate(largest_component)}

# iterate through the df to fill the adjacency matrix
for _, row in only_connected_biogrid.iterrows():
    gene_a_index = gene_to_index.get(row['#ID Interactor A'])
    gene_b_index = gene_to_index.get(row['ID Interactor B'])
    confidence_value = row['Standardized_confidence_values']
    if not np.isnan(confidence_value):
        adj_matrix[gene_a_index, gene_b_index] = confidence_value
        adj_matrix[gene_b_index, gene_a_index] = confidence_value  

# check for rows in the adjacency matrix that have all zeros
rows_with_all_zeros = np.where(~adj_matrix.any(axis=1))[0]

if len(rows_with_all_zeros) > 0:
    print("Rows with all zeros found at indices:", rows_with_all_zeros)
else:
    print("No rows with all zeros found.")
    
connected_biogrid_graph = nx.from_pandas_edgelist(only_connected_biogrid, '#ID Interactor A', 'ID Interactor B')
is_connected = nx.is_connected(connected_biogrid_graph)
print(is_connected)
# output whether the graph is connected or not
if is_connected:
    print("The Human BioGRID network is fully connected.")
else:
    print("The Human BioGRID network is not fully connected.")


False
The Human BioGRID network is not fully connected.
Largest component size: 15800.
No rows with all zeros found.
True
The Human BioGRID network is fully connected.


In [26]:
# read in disgnet disease gene associations, this is base don using the mondo ontology to propagate
disgenet_propagated = pd.read_csv('../data/processed/tb_propagated_disgenet_genes.tsv', delimiter= '\t')
propagated_tb_genes = [str(gene) for gene in set(disgenet_propagated['gene_id'].tolist())]
print(len(propagated_tb_genes))

106


In [27]:
# function that will check to see if the positive genes are in the network. 
# If they are not in the network then they are taken out. Keep track of which genes are removed

def genes_in_network(positive_list, genes_to_index, adjacency_matrix):
    # get the set of genes that are in the new adjacency matrix
    genes_in_network_set = set(genes_to_index.keys())

    removed_genes = []
    
    # keep genes in the positive list if they are found in the network
    filtered_positive_list = []
    for gene in positive_list:
        if gene in genes_in_network_set:
            filtered_positive_list.append(gene)
        else:
            removed_genes.append(gene)
        
     # get the indexes of the the negative genes to get them from the adj matrix
    positive_indices = [genes_to_index[gene] for gene in filtered_positive_list]
    
    # get the negative gene subset from the adjacency matrix 
    positive_adjacency_matrix = adjacency_matrix[positive_indices, :]
    
    # make new gene to index dictionary containing only the negative genes
    positive_gene_to_index = {gene: genes_to_index[gene] for gene in positive_list if gene in genes_to_index}

    # since shape of adj matrix changed, have to start new indexes
    new_positive_indices = list(range(positive_adjacency_matrix.shape[0]))

    # make new gene to index dictionary containing only the negative genes
    new_positive_gene_to_index = {gene: index for index, gene in enumerate(filtered_positive_list)}

    return filtered_positive_list, removed_genes, positive_adjacency_matrix, positive_gene_to_index, new_positive_gene_to_index


In [28]:
tb_pos, removed_disgenet_genes, tb_pos_disgenet_features, disgenet_posgen2ind, new_disgenet_posgen2ind = genes_in_network(propagated_tb_genes, gene_to_index, adj_matrix)
# get some quick counts on genes in and out of network

print(f"Positive genes in network for DisGeNET: {len(tb_pos)}")

print(f"Genes not in network for DisGeNET set: {len(removed_disgenet_genes)}")

print(f"Positve gene tb disgenet list {tb_pos}")
print(f"new pos disgenet gene2indices {new_disgenet_posgen2ind}")
print(f"Positve gene tb disgenet list length {len(tb_pos)}")
print(f"new pos disgenet gene2indices length {len(new_disgenet_posgen2ind)}")

Positive genes in network for DisGeNET: 86
Genes not in network for DisGeNET set: 20
Positve gene tb disgenet list ['3586', '10', '3594', '345611', '1555', '3605', '3606', '27161', '23583', '3105', '1571', '3623', '3627', '3119', '3123', '3125', '3127', '4153', '5199', '7248', '7249', '6737', '51284', '30813', '100', '27241', '54892', '30835', '6774', '50807', '64127', '8838', '2213', '3240', '3242', '181', '59067', '9927', '6347', '717', '6352', '4312', '4318', '4843', '54509', '1786', '7421', '50943', '3329', '10529', '129831', '79659', '820', '9021', '80196', '3918', '847', '54106', '9564', '1380', '3431', '25966', '3439', '6002', '3447', '1401', '7040', '3458', '3459', '11151', '1437', '929', '5034', '114609', '7096', '7097', '27074', '4045', '1999', '7124', '3552', '3558', '8170', '3569', '7153', '3576']
new pos disgenet gene2indices {'3586': 0, '10': 1, '3594': 2, '345611': 3, '1555': 4, '3605': 5, '3606': 6, '27161': 7, '23583': 8, '3105': 9, '1571': 10, '3623': 11, '3627': 12, 

In [29]:
def get_negatives(positive_list, genes_to_index, adjacency_matrix, seed=SEED):
    random.seed(seed)
    np.random.seed(seed)

    # get genes that are in the network
    genes_in_network = set(genes_to_index.keys())

    # exclude positive genes from this universe
    genes_in_network -= set(positive_list)

    # convert set to list
    genes_in_network_list = list(genes_in_network)

    # determine the number of negative genes needed
    num_negative_genes_needed = len(positive_list)

    # filter out negative genes not in the network
    negative_list = [gene for gene in genes_in_network_list if gene in genes_to_index]

    # randomly select negative genes to match the number of positive genes
    negative_list = random.sample(genes_in_network_list, num_negative_genes_needed)

    # get the indexes of the negative genes to retrieve them from the adjacency matrix
    negative_indices = [genes_to_index[gene] for gene in negative_list]
    
    # get the negative gene subset from the adjacency matrix 
    negative_adjacency_matrix = adjacency_matrix[negative_indices, :]

    # create a new gene-to-index dictionary containing only the negative genes
    negative_gene_to_index = {gene: genes_to_index[gene] for gene in negative_list if gene in genes_to_index}

    # since the shape of the adjacency matrix changed, start new indexes
    new_negative_indices = list(range(negative_adjacency_matrix.shape[0]))

    # create a new gene-to-index dictionary containing only the negative genes
    new_negative_gene_to_index = {gene: index for index, gene in enumerate(negative_list)}

    return negative_list, negative_adjacency_matrix, negative_gene_to_index, new_negative_gene_to_index



In [30]:
# get negative list
tb_neg, tb_disgenet_neg_feat, old_disgenet_negind, disgenet_neg_gene2ind = get_negatives(tb_pos, gene_to_index, adj_matrix)

# get quick count to make sure the negatives match the number of positives
print(f"Negative genes in network for DisGeNET: {len(tb_neg)} {tb_neg}")
print(f"adj_matrix for disgenet negatives shape and view: {tb_disgenet_neg_feat.shape} \n{tb_disgenet_neg_feat}")
print(disgenet_neg_gene2ind)

# Write the negative list to a TSV file
negatives_df = pd.DataFrame(tb_neg, columns=["Negative Genes"])
negatives_df.to_csv("../data/processed/tb_negative_genes.tsv", sep='\t', index=False)

Negative genes in network for DisGeNET: 86 ['83853', '210', '11157', '91373', '11041', '90135', '6903', '148268', '84337', '516', '27333', '117145', '148156', '1455', '3815', '26223', '116154', '63970', '55066', '10579', '22913', '381', '222962', '23205', '1652', '5464', '6700', '22994', '9136', '4909', '6504', '64066', '284359', '340547', '813', '9446', '1912', '9252', '155370', '2259', '10174', '51649', '80235', '54935', '23389', '5192', '11213', '55030', '7574', '23294', '376132', '10277', '170850', '5164', '286205', '5777', '55599', '3099', '114327', '55671', '998', '27123', '3275', '654429', '26520', '91608', '1616', '389123', '6834', '401152', '9181', '672', '57560', '284106', '5355', '286016', '23677', '51059', '8886', '1641', '1665', '55859', '55222', '79722', '7434', '51186']
adj_matrix for disgenet negatives shape and view: (86, 15800) 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. 